In [ ]:
import requests
import pandas as pd
from pathlib import Path


In [ ]:
OUT = Path(r"C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\data_raw")
OUT.mkdir(parents=True, exist_ok=True)

In [ ]:
def crawl_worldbank_raw(indicator: str, country: str = "VN", out_dir: Path = OUT):
    """
    Lấy dữ liệu World Bank 'thô': giữ nguyên NaN, không ép kiểu, không sort.
    """
    base = f"https://api.worldbank.org/v2/country/{country}/indicator/{indicator}"
    page, per_page, all_rows = 1, 1000, []

    while True:
        r = requests.get(base, params={"format": "json", "per_page": per_page, "page": page}, timeout=30)
        r.raise_for_status()
        payload = r.json()
        if not isinstance(payload, list) or len(payload) < 2 or payload[1] is None:
            break

        meta, data = payload[0], payload[1]
        if not data: break
        for item in data:
            all_rows.append({
                "indicator_id": (item.get("indicator") or {}).get("id"),
                "indicator_value": (item.get("indicator") or {}).get("value"),
                "country_id": (item.get("country") or {}).get("id"),
                "country_value": (item.get("country") or {}).get("value"),
                "countryiso3code": item.get("countryiso3code"),
                "date": item.get("date"),
                "value": item.get("value"),      
                "unit": item.get("unit"),
                "obs_status": item.get("obs_status"),
                "decimal": item.get("decimal"),
            })
        if page >= meta.get("pages", 1): break
        page += 1

    df = pd.DataFrame(all_rows, columns=[
        "indicator_id","indicator_value","country_id","country_value",
        "countryiso3code","date","value","unit","obs_status","decimal"
    ])
    out_path = out_dir / f"{indicator}_{country}_WB_RAW.csv"
    df.to_csv(out_path, index=False, encoding="utf-8-sig")
    print(f"✅ WorldBank {indicator} ({country}) -> {len(df)} dòng | {out_path}")
    return df

In [ ]:
 # Inflation (annual %)
wb_inflation_pct = crawl_worldbank_raw("FP.CPI.TOTL.ZG", "VN",OUT) 

In [ ]:
# CPI index (2010=100)
wb_cpi_index     = crawl_worldbank_raw("FP.CPI.TOTL",    "VN",OUT)  